# eBay Kleinanzeigen

In this guided project I will analyze data from eBay Kleinanzeigen from car sales

In [1]:
import pandas as pd
import numpy as np
autos = pd.read_csv('autos.csv', encoding = 'Latin-1')

In [2]:
autos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

# Observations about the data
* 50,000 entries
* 12 Columns (most are strings)
* Column names in camelCase not snake_case
* Columns with nan-Objects:
    * vehicle type
    * model
    * fuel type
    * not repaired damage
* Column names and entries are in German, which I assume I need to change

In [3]:
autos_columns = autos.columns
for name in autos_columns:
    name = name.strip()
autos.columns = autos_columns

replace_dic = {'dateCrawled':'date_crawled','offerType' : 'offer_type', 'vehicleType' : 'vehicle_type' ,
               'yearOfRegistration' : 'registration_year', 
               'powerPS' : 'power_ps', 'monthOfRegistration' : 'registration_month'
              , 'fuelType' : 'fuel_type', 'notRepairedDamage' : 'unrepaired_damage'
              , 'dateCreated' : 'ad_created', 'nrOfPictures' : 'nr_of_pictures'
              ,'postalCode' : 'postal_code', 'lastSeen' : 'last_seen'}
autos.rename(columns = replace_dic, inplace = True)



In [4]:
#autos.describe(include = 'all')


In [5]:
print(autos['ad_created'].value_counts())

2016-04-03 00:00:00    1946
2016-03-20 00:00:00    1893
2016-03-21 00:00:00    1886
2016-04-04 00:00:00    1844
2016-03-12 00:00:00    1831
2016-03-14 00:00:00    1761
2016-04-02 00:00:00    1754
2016-03-28 00:00:00    1748
2016-03-07 00:00:00    1737
2016-03-29 00:00:00    1707
2016-03-19 00:00:00    1692
2016-04-01 00:00:00    1690
2016-03-15 00:00:00    1687
2016-03-30 00:00:00    1672
2016-03-08 00:00:00    1667
2016-03-09 00:00:00    1662
2016-03-22 00:00:00    1640
2016-03-11 00:00:00    1639
2016-03-26 00:00:00    1628
2016-03-23 00:00:00    1609
2016-03-31 00:00:00    1596
2016-03-25 00:00:00    1594
2016-03-10 00:00:00    1593
2016-03-17 00:00:00    1560
2016-03-27 00:00:00    1545
2016-03-16 00:00:00    1500
2016-03-24 00:00:00    1454
2016-03-05 00:00:00    1152
2016-03-13 00:00:00     846
2016-03-06 00:00:00     756
                       ... 
2016-01-27 00:00:00       3
2016-02-02 00:00:00       2
2016-02-18 00:00:00       2
2016-01-10 00:00:00       2
2016-02-20 00:00:00 

Columns as candidates for being dropped:
* Seller:
    All the entries are private except for one. Should probabaly exclude that one as well as this column
* Offer Type:
    All are offers, one is an ask. This does not provide much information, should probably exclude the one asking entry
* nr of pictures: all are 0. This is useless and incorrect information
* registration month: no useful information

Columns for cleaning:
* Name, very much not uniform
* Model

Numeric Values stored as strings:
* price
* odometer



In [6]:
# Make price and odometer into numerical values and rename their column names to
# include measurements
print(autos['price'].dtype)
if autos['price'].dtype == object:
    print('true')
    autos['price'] = autos['price'].str.replace('$','')
    autos['price'] = autos['price'].str.replace(',','')
    autos['price'] = autos['price'].astype(float)
    

if autos['odometer'].dtype == object:
    autos['odometer'] = autos['odometer'].str.replace('km','')
    autos['odometer'] = autos['odometer'].str.replace(',','')
    autos['odometer'] = autos['odometer'].astype(int)

#change the names of the columns
print(autos.rename(columns = {'odometer' : 'odometer_km'}, inplace = True))





object
true
None


In [7]:
def explore(name):
    print('min: '+ str(autos[name].min()))
    print('max: ' + str(autos[name].max()))
    print(autos[name].unique().shape)
    print(autos[name].describe())
    print((autos)[name].value_counts().sort_values(axis= 0,ascending = False).head())
explore('odometer_km')

min: 5000
max: 150000
(13,)
count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64
150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
Name: odometer_km, dtype: int64


# Excluding unrealistic values
* Price:
    * maximum: 200k. Nobody is going to sell a brand new supercar on kleinanzeigen
    * minimum 500. Just to exclude the 0 which are often used for other things than to offer a product (ask if someone has a product) and to exclude car parts put put into the car category for more views
    
* Odometer:
    * no minimum, people could be selling new cars also the minimum has been pre-set to 500
    * maximum: 500,000 cars don't last much longer

In [8]:
price_mask = autos['price'].between(1, 200000)
odometer_mask = autos['odometer_km'].between(0,500000)
#print(price_mask & odometer_mask)
#price_mask.shape
autos = autos.loc[price_mask]

#print(autos.loc[autos.loc[(price_mask),:],:].shape())


Now we will analyze the date distribution

In [32]:
print(autos['date_crawled'].str[:10].value_counts(normalize = True, dropna = False).sort_index())

2016-03-05    0.025331
2016-03-06    0.014045
2016-03-07    0.036020
2016-03-08    0.033301
2016-03-09    0.033095
2016-03-10    0.032189
2016-03-11    0.032580
2016-03-12    0.036926
2016-03-13    0.015672
2016-03-14    0.036555
2016-03-15    0.034290
2016-03-16    0.029615
2016-03-17    0.031633
2016-03-18    0.012913
2016-03-19    0.034784
2016-03-20    0.037894
2016-03-21    0.037358
2016-03-22    0.032992
2016-03-23    0.032210
2016-03-24    0.029347
2016-03-25    0.031612
2016-03-26    0.032210
2016-03-27    0.031077
2016-03-28    0.034825
2016-03-29    0.034104
2016-03-30    0.033672
2016-03-31    0.031839
2016-04-01    0.033692
2016-04-02    0.035484
2016-04-03    0.038594
2016-04-04    0.036493
2016-04-05    0.013077
2016-04-06    0.003172
2016-04-07    0.001400
Name: date_crawled, dtype: float64


# Conclusions from date_crawled:

* The crawler was activated every day for a month. It enters the new data.
* There are significant drops in some days in the number of entries on the site. These are not correlated with the days of the week or with German holidays
* It is currently not known why these drops exist and are so pronunced. The drop in entries in the last three days indicates a correlation to the crawler in some way

In [31]:
print(autos['ad_created'].str[:10].value_counts(dropna=False, normalize = True).sort_index(ascending = False))

2016-04-07    0.001256
2016-04-06    0.003254
2016-04-05    0.011801
2016-04-04    0.036864
2016-04-03    0.038841
2016-04-02    0.035155
2016-04-01    0.033692
2016-03-31    0.031880
2016-03-30    0.033486
2016-03-29    0.034042
2016-03-28    0.034949
2016-03-27    0.030974
2016-03-26    0.032271
2016-03-25    0.031756
2016-03-24    0.029285
2016-03-23    0.032045
2016-03-22    0.032807
2016-03-21    0.037564
2016-03-20    0.037955
2016-03-19    0.033692
2016-03-18    0.013592
2016-03-17    0.031283
2016-03-16    0.030130
2016-03-15    0.034022
2016-03-14    0.035196
2016-03-13    0.017011
2016-03-12    0.036761
2016-03-11    0.032910
2016-03-10    0.031901
2016-03-09    0.033157
                ...   
2016-02-21    0.000062
2016-02-20    0.000041
2016-02-19    0.000062
2016-02-18    0.000041
2016-02-17    0.000021
2016-02-16    0.000021
2016-02-14    0.000041
2016-02-12    0.000041
2016-02-11    0.000021
2016-02-09    0.000021
2016-02-08    0.000021
2016-02-07    0.000021
2016-02-05 

# ad_created interpretation

* The data includes timeframes from before the crawler was activated. These show very low numbers, indicating, that the data was taken from the website when the crawler was already active, Thus only showing the ads still active at that time. This also shows that typically the ads are not on the website for over a month.
* In the time where the crawler was active, the number of new ads for a given day is not equal to the ads found by the crawler, this could be because the data from the crawler was taken at a different time than the data from the website

In [30]:
print(autos['last_seen'].str[:10].value_counts(normalize = True, dropna = False).sort_index())

2016-03-05    0.001071
2016-03-06    0.004325
2016-03-07    0.005396
2016-03-08    0.007414
2016-03-09    0.009597
2016-03-10    0.010668
2016-03-11    0.012377
2016-03-12    0.023786
2016-03-13    0.008897
2016-03-14    0.012604
2016-03-15    0.015878
2016-03-16    0.016455
2016-03-17    0.028091
2016-03-18    0.007352
2016-03-19    0.015837
2016-03-20    0.020656
2016-03-21    0.020636
2016-03-22    0.021377
2016-03-23    0.018535
2016-03-24    0.019771
2016-03-25    0.019215
2016-03-26    0.016805
2016-03-27    0.015631
2016-03-28    0.020841
2016-03-29    0.022345
2016-03-30    0.024775
2016-03-31    0.023786
2016-04-01    0.022798
2016-04-02    0.024919
2016-04-03    0.025187
2016-04-04    0.024487
2016-04-05    0.124740
2016-04-06    0.221801
2016-04-07    0.131948
Name: last_seen, dtype: float64


# Interpretation of last_seen:

* This column was created by the crawler. It only includes days on which the crawler was active

* On days on which very few ads were first seen by the crawler, a lot of them were taken off the website. 

In [29]:
print(autos['registration_year'].describe())

count    48557.000000
mean      2004.754968
std         88.651037
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


# Interpretation registration year

* There are several outliers (1000 and 9999)

* On average the cars on the site are 12 years old


# Cleaning Data from the registration year column

* earliest is 1901, because if someone were to enter the wrong registration date, it might be 1900, but not 1901
* latest ist 2016, because this is the year the data was taken and data from time travelers is norotiously unreliable

In [37]:
autos['registration_year'].between(1901,2016).value_counts()


True     46673
False     1884
Name: registration_year, dtype: int64

# Removal of inaccurate registration_year data

This is alright, because we will be removing only 1884 data points

In [39]:
autos = autos.loc[autos.loc[:,'registration_year'].between(1901, 2016),:]
print(autos.shape)

(46673, 20)


# Analysis by car brand

In [44]:
print(autos['brand'].value_counts(normalize = True))

volkswagen        0.211300
bmw               0.110021
opel              0.107600
mercedes_benz     0.096480
audi              0.086581
ford              0.069912
renault           0.047158
peugeot           0.029846
fiat              0.025647
seat              0.018276
skoda             0.016412
nissan            0.015276
mazda             0.015191
smart             0.014162
citroen           0.014012
toyota            0.012705
hyundai           0.010027
sonstige_autos    0.009792
volvo             0.009149
mini              0.008763
mitsubishi        0.008227
honda             0.007842
kia               0.007070
alfa_romeo        0.006642
porsche           0.006021
suzuki            0.005935
chevrolet         0.005699
chrysler          0.003514
dacia             0.002635
daihatsu          0.002507
jeep              0.002271
subaru            0.002143
land_rover        0.002100
saab              0.001650
jaguar            0.001564
daewoo            0.001500
trabant           0.001393
r

# Observations

* The most common car brands are German
* Volkswagen is by a wide margin the most common brand of car
* Cars with only a low market share don't provide enough data
* only car brands with more than 5% mark will be used
    * These brands are all German (German Ford is made in cologne, the models are distinct from the American ones)

In [60]:
mean_price_brand = {}
indices = autos['brand'].value_counts(normalize = True)[:6].index
for index in indices:
    mean_price_brand[index] = autos.loc[autos['brand'] == index,'price'].mean()
print(mean_price_brand)

{'mercedes_benz': 8628.450366422385, 'ford': 3749.4695065890287, 'audi': 9336.687453600594, 'volkswagen': 5402.410261610221, 'opel': 2975.2419354838707, 'bmw': 8236.9423563778}


# Analysis of the data

* The premium brands (Mercedes, Audi, BMW) are more expensive than the exonomy brands (Opel, Ford). Volkswagen is in the middle

# Analysis of 'milage' (in kilometers)

In [64]:
mean_milage_brand = {}
for index in indices:
    mean_milage_brand[index] = autos.loc[autos['brand'] == index,'odometer_km'].mean()
print(mean_milage_brand)

{'mercedes_benz': 130788.36331334666, 'ford': 124266.01287159056, 'audi': 129157.38678544914, 'volkswagen': 128707.15879132022, 'opel': 129310.0358422939, 'bmw': 132619.2794547225}


# Turn them into pandas series

In [85]:
milage = pd.Series(mean_milage_brand)
price = pd.Series(mean_price_brand)
df_comparison = pd.DataFrame(price, columns = ['mean_price'])
df_comparison['mean_milage'] = milage
print(df_comparison)

,mean_price,mean_milage
audi,9336.687454,129157.386785
bmw,8236.942356,132619.279455
ford,3749.469507,124266.012872
mercedes_benz,8628.450366,130788.363313
opel,2975.241935,129310.035842
volkswagen,5402.410262,128707.158791


# Analysis

* The average milage is the same across the board
* People sell cars at a time before they lose too much value
*